###  Convert Annotations to YOLO Format

In [4]:
import cv2
import os

# Convert bounding box to YOLO format
def to_yolo_format(img_w, img_h, x, y, w, h):
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    width = w / img_w
    height = h / img_h
    return [0, x_center, y_center, width, height]  # class_id 0 for face

# Convert WIDER FACE annotations to YOLO format
def convert_widerface_to_yolo(label_path, img_folder, save_img_folder, save_label_folder):
    os.makedirs(save_img_folder, exist_ok=True)
    os.makedirs(save_label_folder, exist_ok=True)

    with open(label_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]  # remove empty lines

    i = 0
    while i < len(lines):
        img_rel_path = lines[i]
        i += 1

        if i >= len(lines):
            print(f"[Warning] Missing face count after image: {img_rel_path}")
            break

        try:
            face_num = int(lines[i])
        except ValueError:
            print(f"[Error] Invalid face count '{lines[i]}' at line {i}, skipping image: {img_rel_path}")
            continue
        i += 1

        full_img_path = os.path.join(img_folder, img_rel_path)
        image = cv2.imread(full_img_path)

        if image is None:
            print(f"[Warning] Cannot read image: {full_img_path}")
            i += face_num  # skip all bounding boxes
            continue

        img_h, img_w = image.shape[:2]

        # Save image to new folder
        img_filename = os.path.basename(img_rel_path)
        save_img_path = os.path.join(save_img_folder, img_filename)
        cv2.imwrite(save_img_path, image)

        # Create label file
        label_filename = os.path.splitext(img_filename)[0] + ".txt"
        label_path = os.path.join(save_label_folder, label_filename)

        with open(label_path, 'w') as label_out:
            for _ in range(face_num):
                if i >= len(lines):
                    print(f"[Warning] Ran out of lines while reading boxes for {img_rel_path}")
                    break
                try:
                    x, y, w, h, *_ = map(float, lines[i].split())
                    i += 1
                except ValueError:
                    print(f"[Error] Invalid bbox format at line {i} for image {img_rel_path}")
                    i += 1
                    continue

                if w <= 0 or h <= 0:
                    continue  # skip invalid boxes

                yolo_box = to_yolo_format(img_w, img_h, x, y, w, h)
                label_out.write(" ".join(map(str, yolo_box)) + "\n")

# Convert training set
convert_widerface_to_yolo(
    label_path="wider_face_data/wider_face_split/wider_face_train_bbx_gt.txt",
    img_folder="wider_face_data/WIDER_train/images",
    save_img_folder="wider_face_yolo/train/images",
    save_label_folder="wider_face_yolo/train/labels"
)

# Convert validation set
convert_widerface_to_yolo(
    label_path="wider_face_data/wider_face_split/wider_face_val_bbx_gt.txt",
    img_folder="wider_face_data/WIDER_val/images",
    save_img_folder="wider_face_yolo/val/images",
    save_label_folder="wider_face_yolo/val/labels"
)


[Error] Invalid face count '0--Parade/0_Parade_Parade_0_630.jpg' at line 10424, skipping image: 0 0 0 0 0 0 0 0 0 0
[Error] Invalid face count '2--Demonstration/2_Demonstration_Political_Rally_2_414.jpg' at line 86539, skipping image: 0 0 0 0 0 0 0 0 0 0
[Error] Invalid face count '39--Ice_Skating/39_Ice_Skating_iceskiing_39_195.jpg' at line 133394, skipping image: 0 0 0 0 0 0 0 0 0 0
[Error] Invalid face count '46--Jockey/46_Jockey_Jockey_46_138.jpg' at line 145714, skipping image: 0 0 0 0 0 0 0 0 0 0


###  Create data.yaml File for YOLOv8



In [5]:
yaml_content = """
path: wider_face_yolo
train: train/images
val: val/images
names:
  0: face
"""

with open("wider_face_yolo/data.yaml", "w") as f:
    f.write(yaml_content)


###  Train YOLOv8 for Face Detection

In [6]:
from ultralytics import YOLO

# Load pretrained YOLOv8n model
model = YOLO("yolov8n.pt")

# Freeze all layers
for name, param in model.model.named_parameters():
    param.requires_grad = False

# Unfreeze the last 10 layers
# This includes detection heads and some backbone layers
unfreeze_count = 10
for name, param in list(model.model.named_parameters())[-unfreeze_count:]:
    param.requires_grad = True
    print(f"[Unfrozen] {name}")

# Train with partial fine-tuning
model.train(
    data="wider_face_yolo/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name="yolov8-face",
    verbose=True
)


100%|██████████| 6.25M/6.25M [00:12<00:00, 526kB/s]


[Unfrozen] model.22.cv3.1.2.bias
[Unfrozen] model.22.cv3.2.0.conv.weight
[Unfrozen] model.22.cv3.2.0.bn.weight
[Unfrozen] model.22.cv3.2.0.bn.bias
[Unfrozen] model.22.cv3.2.1.conv.weight
[Unfrozen] model.22.cv3.2.1.bn.weight
[Unfrozen] model.22.cv3.2.1.bn.bias
[Unfrozen] model.22.cv3.2.2.weight
[Unfrozen] model.22.cv3.2.2.bias
[Unfrozen] model.22.dfl.conv.weight
Ultralytics 8.3.161 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 7787MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=wider_face_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=

100%|██████████| 5.35M/5.35M [00:03<00:00, 1.63MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1708.9±319.3 MB/s, size: 189.1 KB)


train: Scanning /mnt/A04C91DC4C91AE12/Israil/work/DL/Object_Detection/Assignment-7/bbbbbb/wider_face_yolo/train/labels... 12880 images, 4 backgrounds, 0 corrupt: 100%|██████████| 12880/12880 [00:07<00:00, 1776.53it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:26<00:00,  3.87it/s]


                   all       3226      39696      0.841      0.579       0.66      0.361

50 epochs completed in 2.161 hours.
Optimizer stripped from runs/detect/yolov8-face/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8-face/weights/best.pt, 6.2MB

Validating runs/detect/yolov8-face/weights/best.pt...
Ultralytics 8.3.161 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 7787MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:13<00:00,  7.46it/s]


                   all       3226      39696      0.842      0.577      0.659      0.361
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/yolov8-face


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78da807936e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Run Inference

In [2]:
from ultralytics import YOLO

# Load best model after training
model = YOLO("runs/detect/yolov8-face/weights/best.pt")

# Predict on image
model.predict(source="person.jpg", save=True)



image 1/1 /mnt/A04C91DC4C91AE12/Israil/work/DL/Object_Detection/Assignment-7/bbbbbb/person.jpg: 480x640 24 faces, 20.7ms
Speed: 1.7ms preprocess, 20.7ms inference, 92.3ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict3


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'face'}
 obb: None
 orig_img: array([[[160, 154, 155],
         [160, 154, 155],
         [161, 155, 156],
         ...,
         [143, 136, 139],
         [143, 136, 139],
         [143, 136, 139]],
 
        [[160, 154, 155],
         [161, 155, 156],
         [162, 156, 157],
         ...,
         [143, 136, 139],
         [143, 136, 139],
         [143, 136, 139]],
 
        [[162, 156, 157],
         [162, 156, 157],
         [163, 157, 158],
         ...,
         [144, 137, 140],
         [144, 137, 140],
         [144, 137, 140]],
 
        ...,
 
        [[  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1]],
 
        [[  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [  1,   1, 